In [ ]:

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'mine-the-model-2023:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F62583%2F6927485%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240720%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240720T091923Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D9ee95c48e5bf05f84698a4d2307ee7875348aec2e275e2f6ce5175afbb1a98f75c878365902a80e56817b98f689f59fda5f1d7e27bc109054a817ed9d446c1edfc038bdd54eed9c48459cb556a6c44fa10049558c4a5cf04dd97e0ac3aa4c0118317d2eef6054dc41c09e0e7fda6fdae872de31568b17e1c35812475556b819d76a5970f740e4a25bb9ab06630e03962a3e641a42cbf744e0a0c203eac706af080bc851cda63e6298f6d233f06e7aa5b8897f7e40fe4ae9e88f4ab92deab751672a7c2b1c90a2a282839ee34d7cf419fb6703ff03efeae81df524a30904249cc8b3f4e3c336b6fbb8006d167ba42fb1858432a7e6b0b2b4d301bbdca01020d8f'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)

    with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
        total_length = fileres.headers['content-length']
        dl = 0
        data = fileres.read(CHUNK_SIZE)
        while len(data) > 0:
            dl += len(data)
            tfile.write(data)
            done = int(50 * dl / int(total_length))
            sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
            sys.stdout.flush()
            data = fileres.read(CHUNK_SIZE)
        if filename.endswith('.zip'):
          with ZipFile(tfile) as zfile:
            zfile.extractall(destination_path)
        else:
          with tarfile.open(tfile.name) as tarfile:
            tarfile.extractall(destination_path)


In [ ]:
import os
import pandas as pd

import matplotlib.pyplot as plt

from prophet import Prophet

In [ ]:
def predictor(path):
  df = pd.read_csv(path)
  df = df[['Date','Close']]
  df.columns = ['ds','y']
  model = Prophet(daily_seasonality=True)
  model.fit(df)
  prediction_dates = model.make_future_dataframe(periods=100)
  prediction_dates = prediction_dates.tail(100)
  predictions = model.predict(prediction_dates)

  pred_df = predictions[['ds','yhat']]
  pred_df.columns = ['Date','Price']

  ids=[]
  for index, row in pred_df.iterrows():
    ID = f"{os.path.splitext(os.path.basename(path))[0]}_#{index+1}"
    ids.append(ID)

  pred_df['Date'] = ids

  return pred_df


In [ ]:
folder_path = "/kaggle/input/mine-the-model-2023/Upload-Dataset/TRAIN"
dfs=[]

for filename in os.listdir(folder_path):
  stock_df = os.path.join(folder_path,filename)
  dfs.append(predictor(stock_df))

In [ ]:
dfsconcat = pd.concat(dfs)
dfsconcat.columns = ['ID','Price']
dfsconcat.to_csv('Submission_prophet.csv',index=False)

In [ ]:
plt.plot(dfsconcat['ID'][0:100],dfsconcat['Price'][0:100])
plt.title('Closing Price v/s Time',size=20)
plt.xticks(c='white')
plt.xlabel('Date',size=15,)
plt.ylabel('Closing Price',size=15)